In [158]:
import numpy as np
import pandas as pd
import json
import pandasql as psql
import eng_to_ipa as ipa
import matplotlib.pyplot as plt
import seaborn as sns

In [159]:
movies_df = pd.read_csv('MovieSummaries/movie.metadata.tsv',sep='\t',names=['Wikipedia_ID','Freebase_ID','Name','Release_date','Box_office_revenues','Runtime','Languages','Country','Genres'])
#movies_df.head()

character_df = pd.read_csv('MovieSummaries/character.metadata.tsv',sep='\t', names=['Wikipedia_ID','Freebase_ID','Release_date','Character_name','Actor_DOB',
                                                                                    'Sex','Height','Ethnicity','Actor_name','Actor_age','Freebase_character_map'
                                                                                    ,'?','??'])

plot_summaries = pd.read_csv('MovieSummaries/plot_summaries.txt', sep='\t', names=['Wikipedia_ID', 'Synopses'])

In [160]:
# Cleaning Languages, Country and Genres Columns

def str_dict_to_values(dict_in_str: str)->list[str]:
    
    if dict_in_str is None:  
        return []
    dict_ = json.loads(dict_in_str)
    values = list(dict_.values())
    return values

movies_df['Languages'] = movies_df['Languages'].apply(str_dict_to_values)
movies_df['Country'] =movies_df['Country'].apply(str_dict_to_values)
movies_df['Genres'] =movies_df['Genres'].apply(str_dict_to_values)

Exception ignored in: <function tqdm.__del__ at 0x15806bb00>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x15806bb00>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception igno

In [161]:
# Merging the 2 datasets and keeping only interesting features

phonetic_df = pd.merge(movies_df,character_df, on="Wikipedia_ID",how="inner")[['Wikipedia_ID','Name','Languages','Country','Genres','Character_name']]

In [162]:
phonetic_df['Languages'] = phonetic_df['Languages'].astype(str).str.replace(r"[\[\]']", "", regex=True)
phonetic_df['Country'] = phonetic_df['Country'].astype(str).str.replace(r"[\[\]']", "", regex=True)
phonetic_df['Genres'] = phonetic_df['Genres'].astype(str).str.replace(r"[\[\]']", "", regex=True)

phonetic_df.head()

,Wikipedia_ID,Name,Languages,Country,Genres,Character_name
0,975900,Ghosts of Mars,English Language,United States of America,"Thriller, Science Fiction, Horror, Adventure, ...",Akooshay
1,975900,Ghosts of Mars,English Language,United States of America,"Thriller, Science Fiction, Horror, Adventure, ...",Lieutenant Melanie Ballard
2,975900,Ghosts of Mars,English Language,United States of America,"Thriller, Science Fiction, Horror, Adventure, ...",Desolation Williams
3,975900,Ghosts of Mars,English Language,United States of America,"Thriller, Science Fiction, Horror, Adventure, ...",Sgt Jericho Butler
4,975900,Ghosts of Mars,English Language,United States of America,"Thriller, Science Fiction, Horror, Adventure, ...",Bashira Kincaid


In [163]:
missing_values_movies = movies_df.isnull().sum()
missing_values_character = character_df.isnull().sum()

def remove_nan_rows(df:pd.DataFrame,column:str)->pd.DataFrame:
    no_nan_df = df.dropna(subset=[column])
    return no_nan_df

character_df = remove_nan_rows(character_df,'Character_name')

phonetic_df = remove_nan_rows(phonetic_df,'Character_name')

In [164]:
import nltk
nltk.download('words')
from nltk.corpus import words

english_words = set(words.words())                                                        # Liste de 235892 mots anglais   
invalid_chars = ['-', '/', '.', ',',"'", '"', '#',"(",')','0','1','2','3','4','5','6','7','8','9']

def filter_non_english_names(name):
    words_in_name = name.split()                                                             # Sépare le nom en mots individuels
    filtered_words = [word for word in words_in_name if word.lower() not in english_words and all(char not in word for char in invalid_chars)]   # Filtre les mots qui ne sont pas dans la liste des mots anglais et ceux qui contiennent '-' ou '/'
    return ' '.join(filtered_words)  


[nltk_data] Error loading words: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


In [165]:
character_df['Character_name']=character_df['Character_name'].apply(filter_non_english_names)

values_filtered = character_df['Character_name'].value_counts()

kept_names = character_df[character_df['Character_name']!=values_filtered.index[0]]

def keep_first_name(name:str)->str:
    return name.split()[0] if name else ""

kept_names['Character_name'] = character_df['Character_name'].apply(keep_first_name)

kept_names.head()

/var/folders/9x/ml3bdy7s7pv3np54mskcntfm0000gn/T/ipykernel_56693/3956894000.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kept_names['Character_name'] = character_df['Character_name'].apply(keep_first_name)


,Wikipedia_ID,Freebase_ID,Release_date,Character_name,Actor_DOB,Sex,Height,Ethnicity,Actor_name,Actor_age,Freebase_character_map,?,??
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Melanie,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


In [181]:
training_data = plot_summaries.sample(500, random_state=42)



from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm import tqdm

sia = SentimentIntensityAnalyzer()
res = {}

for i, row in tqdm(training_data.iterrows(), total=len(training_data)):
    text = row["Synopses"]
    myid = row["Wikipedia_ID"]
    res[myid] = sia.polarity_scores(text)


100%|██████████| 500/500 [00:00<00:00, 653.67it/s]


In [185]:
vaders = pd.DataFrame.from_dict(res, orient='index')

vaders = vaders.reset_index().rename(columns={"index": "Wikipedia_ID"})
vaders = vaders.merge(training_data, how= "left")
vaders.head()


,Wikipedia_ID,neg,neu,pos,compound,Synopses
0,22333286,0.030,0.919,0.051,0.2500,"Set in the 1850s South, the film follows Cassy..."
1,19074986,0.067,0.741,0.192,0.9300,"I.S.A.A.C. , played by former UFC Middleweight..."
2,31193551,0.171,0.739,0.090,-0.9767,"Indrani, a daughter of a multimillionaire Seth..."
3,28631198,0.098,0.814,0.087,-0.4588,Ram is a restaurant owner in Bangkok. Ram lea...
4,27386187,0.073,0.869,0.058,-0.8126,The film starts with the soldiers' last days i...


## Plot VADER results

In [ ]:
sns.barplot(data=vaders)